## Task 1

In [13]:
class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        self.domains = {
            var: set(range(1, 10)) if grid[var[0]][var[1]] == 0 else {grid[var[0]][var[1]]}
            for var in self.variables
        }
        self.constraints = self.generate_constraints()

    def generate_constraints(self):
        """
        Add row column constraints and subgrid constraints
        """
        constraints = []

        return constraints

    def is_valid(self, var, value):
        """
        Check if assigning 'value' to 'var' satisfies all constraints.
        """
        # TODO: Check against other variables in the same constraint group

        row, col = var

        # Check row and column uniqueness
        for i in range(9):
            if self.grid[row][i] == value or self.grid[i][col] == value:
                return False

        # Check 3x3 subgrid uniqueness
        subgrid_row, subgrid_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(3):
            for j in range(3):
                if self.grid[subgrid_row + i][subgrid_col + j] == value:
                    return False

        return True


    def forward_checking(self, var, value):
        """
        Remove 'value' from the domain of neighbors of 'var'.
        Return list of affected variables to restore later.
        """
        affected = []

        row, col = var

        # Eliminate value from domains of neighbors (same row, column, and subgrid)
        for i in range(9):
            for j in range(9):
                if (i == row or j == col or (i // 3 == row // 3 and j // 3 == col // 3)) and self.grid[i][j] == 0:
                    if value in self.domains[(i, j)]:
                        self.domains[(i, j)].remove(value)
                        affected.append((i, j))

        return affected

    def restore_domains(self, affected, value):
        """
        Restore removed values after backtracking.
        """

        for var in affected:
          self.domains[var].add(value)

    def select_unassigned_variable(self):
        # Apply MRV heuristic: choose the variable with the fewest remaining legal values
        unassigned = [var for var in self.variables if self.grid[var[0]][var[1]] == 0]
        return min(unassigned, key=lambda var: len(self.domains[var]))

    def backtrack(self):
        """
        Solve the puzzle using backtracking + MRV + forward checking.
        """
        # Base case: If all cells are assigned, the puzzle is solved
        if all(self.grid[row][col] != 0 for row, col in self.variables):
            return True

        # Select the unassigned variable with the fewest legal values (MRV)
        var = self.select_unassigned_variable()
        row, col = var

        # Try all possible values for this variable
        for value in list(self.domains[var]):
            if self.is_valid(var, value):
                # Assign the value to the grid
                self.grid[row][col] = value

                # Apply forward checking
                affected = self.forward_checking(var, value)

                # Recurse to the next variable
                if self.backtrack():
                    return True

                # Undo the assignment if the recursion didn't succeed
                self.grid[row][col] = 0
                self.restore_domains(affected, value)

        return False

    def solve(self):
        if self.backtrack():
            return self.grid
        else:
            return None

In [14]:

# 9x9 Sudoku Grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solver = SudokuSolver(sudoku_grid)
solution = solver.solve()

# Print the solution
if solution:
    print("Solved Sudoku:")
    for row in solution:
        print(row)
else:
    print("No solution exists.")


Solved Sudoku:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
